<a href="https://colab.research.google.com/github/SandyAtencio/Cleaning_data_Python_Part_1/blob/master/Cleaning_data_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **⚠️😓Problemas comunes de datos**
A continuacion vamos a trabajar sobre un dataset que contiene datos sucios, al cual le vamos a aplicar tecnicas de limpieza como:

✔️Conversion de tipos de datos<br>
✔️Eliminacion de datos duplicados para evitar el doble conteo<br>
✔️Restrinccion de rangos para evitar datos futuros incoherentes

**🧾Informacion del Dataset**<br>
El dataset que estaremos trabajando comprende informacion sobre las estaciones de inicio y finalización, la duración del viaje y cierta información para el usuario de un servicio de bicicletas compartidas.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
df_ride = pd.read_csv("ride_sharing_new.csv")
#veamos que tiene el dataset
df_ride.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ride_id          76 non-null     int64 
 1   duration         76 non-null     object
 2   station_A_id     76 non-null     int64 
 3   station_A_name   76 non-null     object
 4   station_B_id     76 non-null     int64 
 5   station_B_name   76 non-null     object
 6   bike_id          76 non-null     int64 
 7   user_type        76 non-null     int64 
 8   user_birth_year  76 non-null     int64 
 9   user_gender      76 non-null     object
 10  tire_sizes       76 non-null     int64 
 11  ride_date        76 non-null     object
dtypes: int64(7), object(5)
memory usage: 7.2+ KB


La columna `user_type` contiene información sobre si un usuario está viajando gratis y toma los siguientes valores:

🚴‍♂️ `1` de viaje gratis <br>
🚴‍♂️ `2` de pago por viaje <br>
🚴‍♂️ `3` de suscriptores mensuales <br>

In [ ]:
#utilizamos describe para ver el resumen estadistico de los datos de la columna user_type
df_ride['user_type'].describe()

count    76.000000
mean      1.881579
std       0.692187
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       3.000000
Name: user_type, dtype: float64

👁️‍🗨️ Al observar las estadisticas resumidas de la columna `user_type` vemos que  tiene un conjunto finito de valores posibles que representan agrupaciones de datos, por ende, debemos convertir esta columna a tipo categoría.

In [ ]:
#podemos a cambiar el tipo de dato con astype, y se lo asignamos a la nueva columna
df_ride['user_type'] = df_ride['user_type'].astype('category')

#Para asegurarnos que la conversion fue correcta utilizamos la declaracion assert
assert df_ride['user_type'].dtype == 'category'

#y nuevamente observamos las estadisticas resumidas de la nueva columna, y podemos ver que
#el resultado cambia al tratarse de una variable categorica
print(df_ride['user_type'].describe())

count     76
unique     3
top        2
freq      39
Name: user_type, dtype: int64


🧹Ahora bien, siguiendo con nuestra limpieza debemos evitar la inconsistencia de tipo de datos, es decir, debemos asesorarnos que los datos de tipo numericos sean numericos y asi sucesivamente con los demas datos. Esto se hace con el fin de que las operaciones matematicas que querramos realizar sobre esos datos arrojen resultados correctos y no erroneos.

Despues de haber revisado que contiene nuestro dataset, vamos a convertir la columna `durational` que es de tipo `object` a tipo `int`. Sin embargo, antes de eso, debemos asegurarnos de eliminar la palabra "minutes" para asi poderla convertir a numerica.

In [ ]:
#eliminamos la cadena minutes con el metodo strip 
df_ride['duration'] = df_ride['duration'].str.strip('minutes')

#convertirmos duration a tipo int 
df_ride['duration'] = df_ride['duration'].astype('int')

#Para asegurarnos que la conversion fue correcta utilizamos la declaracion assert
assert df_ride['duration'].dtype == 'int'

#Finalmente conocemos cual es la media de la columna duration_time
print(df_ride['duration'].mean())

12.776315789473685


Ahora vamos a trabajar sobre la columna `tire_size` la cual contiene datos sobre el tamaño de los neumáticos de cada bicicleta. el tamaño de los neumáticos de las bicicletas pueden ser 26" 27" y 29" y son almacenados con un valor float.

Actualmente se quieren reducir los costos de mantenimiento, por consiguiente el proveedor decidió establecer el tamaño máximo de neumáticos en 27. Respecto a esto  vamos a asegurarnos de que la columna  `tire_sizes` tenga el rango correcto para esto establecemos el nuevo limite superior 27" para los tamaños de neumáticos, y despues convirtimos la columna a tipo categoria.

In [ ]:
#usamos loc para seleccionar todo esos registros mayores a 27 establecerle el limite superior que es 27
df_ride.loc[df_ride['tire_sizes'] > 27, 'tire_sizes'] = 27
df_ride.head(10)

#convetirmos a tipo categorico
df_ride['tire_sizes'] = df_ride['tire_sizes'].astype('category')

df_ride['tire_sizes'].describe()

count     76
unique     2
top       27
freq      73
Name: tire_sizes, dtype: int64

📅 Ahora procederemos a encontrar todos los registros de la columna `ride_date` que tengan fechas futura, en caso de que si exista esta inconsistencia, lo que haremos sera establecer el valor maximo de esta columna con la fecha de hoy, pero antes de hacer eso debemos convetir a `ride_data` a tipo datetime.

In [ ]:
#convertimos a ride_date en tipo de dato datetime
df_ride['ride_date'] = pd.to_datetime(df_ride['ride_date']).dt.date

#almacenamos la fecha de hoy
today = dt.date.today()

# y por ultimo le aplicamos la fecha de hoy almacenada en today a aquellos registros con
#fechas futuras, es decir con fechas superiores a hoy
df_ride.loc[df_ride['ride_date'] > today, 'ride_date'] = today

#mostramos la fecha maxima
print(df_ride['ride_date'].max())

2021-06-08


LLego la hora, de encontrar los datos duplicados.
Vamos a buscar que registros se encuentran duplicados en nuestro dataset.


In [ ]:
# Aqui buscamos duplicados 
duplicates = df_ride.duplicated('ride_id', keep = False)

# ordenamos por la columna ride_id los registros duplicados obtenidos
df_ride = df_ride[duplicates].sort_values( by = 'ride_id')

# y por ultimo imprimimos los datos duplicados, mostramos 
#en cuales columnas estaban estos registros
print(df_ride[['ride_id','duration','user_birth_year']])

    ride_id  duration  user_birth_year
21       33        11             1986
38       33         6             1966
52       55        13             1999
64       55        10             1999
72       71        11             1987
73       71        11             1987
74       89         9             1990
75       89         9             1990


Despues de conocer los duplicados y las columnas que los almacenan hemos podido observar que hay filas duplicadas tanto completas como incompletas para algunos valores de la columna `ride_id`, con valores diferentes para las columnas `user_birth_year` y duration.

Entonces, lo que vamos hacer es primero tratar esas filas duplicadas eliminando primero los duplicados completos y luego fusionando las filas duplicadas incompletas en una, manteniendo el promedio de `duration` y el mínimo `user_birth_year` para cada conjunto de filas duplicadas incompletas.

In [ ]:
# Eliminamos los duplicados completos en ride_sharing
ride_dup = df_ride.drop_duplicates()

# Creamos un diccionario de estadisticas para funciones de agregacion
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Agrupamos por ride_id y calculamos las nuevas estadisticas
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# nuevamente buscamos valores duplicados
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# y por ultimo nos aseguramos que se haya aplicado todo correctamente
assert duplicated_rides.shape[0] == 0

### **❌🔡Problemas de texto y datos categóricos**

En este espacio, vamos a corregir las incoherencias de hay con espacios en blanco y mayúsculas en las etiquetas de las categorías, tambien, vamos a contraer varias categorías en una sola y cambiar el formato de las cadenas para mantener la coherencia

**🧾Informacion del Dataset**<br>

En este espacio trabajaremos con un dataset llamado `df_airlines` que contiene las respuestas de una encuesta que se le realizaron a los clientes de las aerolineas del aeropuerto de San Francisco. Este dataset tiene metadatos de vuelos, destinos, tiempos de espera, y tambien respuestas a preguntas de seguridad y sastifaccion

Tambien trabajaremos con un dataset llamado `df_categories`, que contiene todos los valores posibles correctos para las columnas de la encuesta.

✔️Utilizaremos ambos dataset para encontrar respuestas de encuestas con valores inconsistentes y eliminarlas, con el fin de poder realizar de manera efectiva una combinación externa e interna en ambos dataset.

In [ ]:
df_categories = pd.read_csv('categories.csv')
df_airlines = pd.read_csv('airlines_final.csv')

#conozcamos cada una de las categorias
print(df_categories)

      cleanliness           safety          satisfaction
0           Clean          Neutral     Very satisfaction
1         Average        Very safe               Neutral
2  Somewhat clean    Somewhat safe    Somewhat satisfied
3  Somewhat dirty      Very unsafe  Somewhat unsatisfied
4          Dirty   Somewhat unsafe      Very unsatisfied


In [ ]:
#Imprimimos los valores unicos de cada columna de la encuesta airlines
#usando el metodo .unique()
print('Cleanliness: ', df_airlines['cleanliness'].unique(), "\n")
print('Safety: ', df_airlines['safety'].unique(), "\n")
print('Satisfaction: ', df_airlines['satisfaction'].unique(), "\n")

Cleanliness:  ['Clean' 'Average' 'Unacceptable' 'Somewhat clean' 'Somewhat dirty'
 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'
 'Very unsatisfied'] 



👁️‍🗨️ Despues de observar lo que contiene cada una de las categorias, hemos detectado que la columna `Cleanliness` de `df_airlines` tiene una categoria inconsistentes que es ***Unacceptable***

👷‍♀️ Ahora vamos a crear un conjunto de la columna `cleanliness` en `df_airlines` y encontremos la categoria inconsistente encontrando la **diferencia** en la columna `cleanliness` de `df_categories`.

In [ ]:
# Aplicamos la diferencia
cat_clean = set(df_airlines['cleanliness']).difference(df_categories['cleanliness'])

#Buscamos filas de df_airlines con un valor en cleanliness
#que no este en df_categories
cat_clean_rows = df_airlines['cleanliness'].isin(cat_clean)

# Imprimimos las filas con categorias inconsistentes
print(df_airlines[cat_clean_rows])

# Imprimimos las filas solo con categorias consistentes
print(df_airlines[~cat_clean_rows])

        id        day  ...     safety        satisfaction
4     2992  Wednesday  ...  Very safe  Somewhat satsified
586   2920   Thursday  ...    Neutral    Very unsatisfied
1816  2439  Wednesday  ...    Neutral             Neutral

[3 rows x 12 columns]
        id       day  ...         safety        satisfaction
0     1351   Tuesday  ...        Neutral      Very satisfied
1      373    Friday  ...      Very safe      Very satisfied
2     2820  Thursday  ...  Somewhat safe             Neutral
3     1157   Tuesday  ...      Very safe  Somewhat satsified
5      634  Thursday  ...      Very safe      Very satisfied
...    ...       ...  ...            ...                 ...
2804  1475   Tuesday  ...        Neutral  Somewhat satsified
2805  2222  Thursday  ...      Very safe      Very satisfied
2806  2684    Friday  ...      Very safe      Very satisfied
2807  2549   Tuesday  ...  Somewhat safe      Very satisfied
2808  2162  Saturday  ...      Very safe  Somewhat satsified

[2474 rows x

💡 Vamos a examinar las columnas categóricas `dest_region` y `dest_size` del dataset `df_airlines`, y nos aseguraremos de que estén limpias y listas para el análisis.

In [ ]:
# Imprimimos los valores unicos de ambas columnas
print('Dest Region: ', df_airlines['dest_region'].unique(), "\n")
print('Dest Size: ', df_airlines['dest_size'].unique(), "\n")

Dest Region:  ['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east'] 

Dest Size:  ['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     '] 



Del resultado anterior podemos concluir los siguientes problemas:

⚠️La columna `dest_region` tiene valores inconsistentes debido a las mayúsculas y tiene un valor que debe reasignarse.

⚠️La columna `dest_size` solo tiene valores inconsistentes debido a los espacios iniciales y finales.

Para resolver los problemas que hemos dectetado procederemos a:<br>
⭐Cambiar las mayusculas de todos los valores de `dest_region` a minisculas.<br>
⭐Remplazar `'eur'` con `'europe'` en `dest_region` usando el metodo `.replace()` <br>
⭐Quitar los espacios en blancos de la columna `dest_size` usando el metodo `.strip()`<br>
⭐Verificar que los cambios se hayan realizado con exito imprimiendo los valores unico de las columnas usando el metodo `.unique()`<br>

In [ ]:
df_airlines['dest_region'] = df_airlines['dest_region'].str.lower()

df_airlines['dest_region'] = df_airlines['dest_region'].replace({'eur':'europe'})

df_airlines['dest_size'] = df_airlines['dest_size'].str.strip()

print('Dest Region: ', df_airlines['dest_region'].unique(), "\n")
print('Dest Size: ', df_airlines['dest_size'].unique(), "\n")

Dest Region:  ['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand'] 

Dest Size:  ['Hub' 'Small' 'Medium' 'Large'] 



Para entender mejor a los encuestado, deseamos averiguar si existe una relación entre ciertas respuestas y el día de la semana y el tiempo de espera en la puerta.

el dataset `df_airlines` contiene las columnas `day` y `wait_min`, que son categóricas y numéricas respectivamente. La columna `day` contiene el día exacto en que se realizó un vuelo y `wait_min` la cantidad de minutos que los viajeros tardaron en esperar en la puerta de embarque. Para facilitar su análisis, deseamos crear dos nuevas variables categóricas:

`wait_type:` 
* `'short'` de 0 a 60 min 
* `'medium'`de 60 a 180 min 
* `'long'` mas de 180 min

`day_week:`
* `'weekday'`si el día está en el día de la semana
* `'weekend'`si el día es en el fin de semana.

Para resolverlo realizaremos los siguientes pasos:

👍 Creamos los rangos y etiquetas para la columna `wait_type`<br>
👍 Creamos la columna `wait_type` desde `wait_min` usando el metodo `pd.cut()`<br>
👍 Creamos el diccionario `mapping` que ayudara a mapear a los dias de la semana `'weekday'` y a los dias del fin de semana`'weekend'`<br>
Creamos la columna `day_week` usando `.replace()`

In [ ]:
# Creamos los rangos para categoria
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Creamos la columna wait_type  
df_airlines['wait_type'] = pd.cut(df_airlines['wait_min'], bins =label_ranges, 
                                labels = label_names)

# Creamos el diccionario
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

df_airlines['day_week'] = df_airlines['day'].replace(mappings)

In [ ]:
df_airlines.head()

,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,day_week
0,1351,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,weekday
1,373,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,weekday
2,2820,Thursday,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium,weekday
3,1157,Tuesday,SOUTHWEST,LOS ANGELES,west us,Hub,Gates 20-39,2018-12-31,190.0,Clean,Very safe,Somewhat satsified,long,weekday
4,2992,Wednesday,AMERICAN,MIAMI,east us,Hub,Gates 50-59,2018-12-31,559.0,Unacceptable,Very safe,Somewhat satsified,long,weekday


Y finalmente nuestros datos han quedado limpios y listos para ser utilizados. 🎉🎉🎉